### Module Mount & Data Load

In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import shutil
import tensorflow as tf

from keras.layers.advanced_activations import LeakyReLU
from math import cos, sin, pi
from PIL import Image
from tqdm import tqdm
from tensorflow.keras import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

In [ ]:
os.chdir('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/data') 

In [ ]:
train = pd.read_csv('./train.csv')
valid = pd.read_csv('./valid.csv')
display(train.head(2))
print()
display(valid.head(2))

In [ ]:
# train.isnull().sum()

In [ ]:
train_paths = glob.glob('./train/*.jpg')
valid_paths = glob.glob('./valid/*.jpg')
test_paths = glob.glob('./test_imgs/*.jpg')
print(len(train_paths), len(valid_paths), len(test_paths))

In [ ]:
train_paths.sort()
valid_paths.sort()

### Train, Valid Split

In [ ]:
# root_dir = '/content/drive/My Drive/Colab/Dacon/Motion_Keypoint/data'

# os.makedirs(root_dir +'/train')
# os.makedirs(root_dir +'/val')

src = "train_imgs"
all_filename = os.listdir(src)
valid_filename = random.sample(all_filename, int(len(train_all) * 0.1))
train_filename = [x for x in all_filename if x not in valid_filename]

print(len(train_filename), len(valid_filename))

train_filename = [src+'/'+ name for name in train_filename]
valid_filename = [src+'/' + name for name in valid_filename]

print('Total images: ', len(all_filename))
print('Training: ', len(train_filename))
print('Validation: ', len(valid_filename))

# Copy-pasting images
for name in tqdm(train_filename):
    shutil.copy(name, 'train')

for name in tqdm(valid_filename):
    shutil.copy(name, 'val')

In [ ]:
train_filename = []
valid_filename = []

for t_paths in tqdm(train_paths):
    filename = t_paths.split('/')[-1]
    train_filename.append(filename)

for v_paths in tqdm(valid_paths):
    filename = v_paths.split('/')[-1]
    valid_filename.append(filename)

In [ ]:
train_df = train[train['image'].isin(train_filename)]
train_df.reset_index(inplace=True, drop=True)

valid_df = train[train['image'].isin(valid_filename)]
valid_df.reset_index(inplace=True, drop=True)

train_df.to_csv('train.csv', index=False)
valid_df.to_csv('valid.csv', index=False)

### 시각화

In [ ]:
plt.figure(figsize=(40,20))
count=1

for i in np.random.randint(0,len(train_paths),5):
    
    plt.subplot(5,1, count)
    
    img_sample_path = train_paths[i]
    img = Image.open(img_sample_path)
    img_np = np.array(img)

    keypoint = train.iloc[:,1:49] #위치 키포인트 하나씩 확인
    keypoint_sample = keypoint.iloc[i, :]
    
    for j in range(0,len(keypoint.columns),2):
        plt.plot(keypoint_sample[j], keypoint_sample[j+1],'rx')
        plt.imshow(img_np)
    
    count += 1

In [ ]:
train['path'] = train_paths
valid['path'] = valid_paths
print(len(train['path']))
print(len(valid['path']))

### Augmentation

In [ ]:
def left_right_flip(images, keypoints):
    flipped_keypoints = []
    flipped_images = np.flip(images, axis=1)
    for idx, sample_keypoints in enumerate(keypoints):
        if idx%2 == 0:
            flipped_keypoints.append(480.-sample_keypoints)
        else:
            flipped_keypoints.append(sample_keypoints)
    return flipped_images, flipped_keypoints

In [ ]:
# shift value setting
pixel_shifts = [12]

def shift_images(images, keypoints):
    # tensor -> numpy
    images = images.numpy()
    shifted_images = []
    shifted_keypoints = []
    for shift in pixel_shifts:    # Augmenting over several pixel shift values
        for (shift_x,shift_y) in [(-shift,-shift),(-shift,shift),(shift,-shift),(shift,shift)]:
            # 이동할 matrix 생성
            M = np.float32([[1,0,shift_x],[0,1,shift_y]])
            shifted_keypoint = np.array([])
            shifted_x_list = np.array([])
            shifted_y_list = np.array([])
            # 이미지 이동
            shifted_image = cv2.warpAffine(images, M, (480,270), flags=cv2.INTER_CUBIC)
            # 이동한만큼 keypoint 수정
            for idx, point in enumerate(keypoints):
                if idx%2 == 0: 
                    shifted_keypoint = np.append(shifted_keypoint, point+shift_x)
                    shifted_x_list = np.append(shifted_x_list, point+shift_x)
                else: 
                    shifted_keypoint =np.append(shifted_keypoint, point+shift_y)
                    shifted_y_list = np.append(shifted_y_list, point+shift_y)
            # 수정된 keypoint가 이미지 사이즈를 벗어나지 않으면 append
            if np.all(0.0<shifted_x_list) and np.all(shifted_x_list<480) and np.all(0.0<shifted_y_list) and np.all(shifted_y_list<270):
                shifted_images.append(shifted_image.reshape(270,480,3))
                shifted_keypoints.append(shifted_keypoint)

    # shifted_keypoints = np.clip(shifted_keypoints,0.0,480)
    return shifted_images, shifted_keypoints

In [ ]:
rotation_angles = [12]

def rotate_augmentation(images, keypoints):
    # tensor -> numpy
    images = images.numpy()
    rotated_images = []
    rotated_keypoints = []
    
    for angle in rotation_angles:
        for angle in [angle,-angle]:
            # 회전할 matrix 생성
            M = cv2.getRotationMatrix2D((240,135), angle, 1.0)
            # cv2_imshow로는 문제없지만 추후 plt.imshow로 사진을 확인할 경우 black screen 생성...
            # 혹시 몰라 matrix를 ndarray로 변환
            M = np.array(M, dtype=np.float32)
            angle_rad = -angle*pi/180
            rotated_image = cv2.warpAffine(images, M, (480,270))
            rotated_images.append(rotated_image)
            
            # keypoint를 copy하여 forloop상에서 값이 계속 없데이트 되는 것을 회피
            rotated_keypoint = keypoints.copy()
            rotated_keypoint[0::2] = rotated_keypoint[0::2] - 240
            rotated_keypoint[1::2] = rotated_keypoint[1::2] - 135
            
            for idx in range(0,len(rotated_keypoint),2):
                rotated_keypoint[idx] = rotated_keypoint[idx]*cos(angle_rad)-rotated_keypoint[idx+1]*sin(angle_rad)
                rotated_keypoint[idx+1] = rotated_keypoint[idx]*sin(angle_rad)+rotated_keypoint[idx+1]*cos(angle_rad)

            rotated_keypoint[0::2] = rotated_keypoint[0::2] + 240
            rotated_keypoint[1::2] = rotated_keypoint[1::2] + 135
            rotated_keypoints.append(rotated_keypoint)
        
    return rotated_images, rotated_keypoints

### Generator

In [ ]:
def trainGenerator():
    # 원본 이미지 resize
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i]) # path(경로)를 통해 이미지 읽기
        img = tf.image.decode_jpeg(img, channels=3) # 경로를 통해 불러온 이미지를 tensor로 변환
        img = tf.image.resize(img, [270,480]) # 이미지 resize 
        img = img/255
        target = train.iloc[:,1:49].iloc[i,:] # keypoint 뽑아주기
        target = target/4

        yield (img, target)
    
    # horizontal flip
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i]) # path(경로)를 통해 이미지 읽기
        img = tf.image.decode_jpeg(img, channels=3) # 경로를 통해 불러온 이미지를 tensor로 변환
        img = tf.image.resize(img, [270,480]) # 이미지 resize 
        img = img/255
        target = train.iloc[:,1:49].iloc[i,:] # keypoint 뽑아주기
        target = target/4
        img, target = left_right_flip(img, target)
        
        yield (img, target)

    # Horizontal & Vertical shift
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270,480])
        img = img/255
        target = train.iloc[:,1:49].iloc[i,:]
        target = target/4
        img_list, target_list = shift_images(img, target)
        for shifted_img, shifted_target in zip(img_list, target_list):
            
            yield (shifted_img, shifted_target)

    # Rotation
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270,480])
        img = img/255
        target = train.iloc[:,1:49].iloc[i,:]
        target = target/4
        img_list, target_list = rotate_augmentation(img, target)
        for rotated_img, rotated_target in zip(img_list, target_list):
            
            yield (rotated_img, rotated_target)

In [ ]:
def validGenerator():
    # 원본 이미지 resize
    for i in range(len(valid)):
        img = tf.io.read_file(valid['path'][i]) # path(경로)를 통해 이미지 읽기
        img = tf.image.decode_jpeg(img, channels=3) # 경로를 통해 불러온 이미지를 tensor로 변환
        img = tf.image.resize(img, [270,480]) # 이미지 resize 
        img = img/255
        target = valid.iloc[:,1:49].iloc[i,:] # keypoint 뽑아주기
        target = target/4

        yield (img, target)
    
    # Horizontal flip
    for i in range(len(valid)):
        img = tf.io.read_file(valid['path'][i]) # path(경로)를 통해 이미지 읽기
        img = tf.image.decode_jpeg(img, channels=3) # 경로를 통해 불러온 이미지를 tensor로 변환
        img = tf.image.resize(img, [270,480]) # 이미지 resize 
        img = img/255
        target = valid.iloc[:,1:49].iloc[i,:] # keypoint 뽑아주기
        target = target/4
        img, target = left_right_flip(img, target)
        
        yield (img, target)

    # Horizontal & Vertical shift
    for i in range(len(valid)):
        img = tf.io.read_file(valid['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270,480])
        img = img/255
        target = valid.iloc[:,1:49].iloc[i,:]
        target = target/4
        img_list, target_list = shift_images(img, target)
        for shifted_img, shifted_target in zip(img_list, target_list):
            
            yield (shifted_img, shifted_target)

    # Rotation
    for i in range(len(valid)):
        img = tf.io.read_file(valid['path'][i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270,480])
        img = img/255
        target = valid.iloc[:,1:49].iloc[i,:]
        target = target/4
        img_list, target_list = rotate_augmentation(img, target)
        for rotated_img, rotated_target in zip(img_list, target_list):
            
            yield (rotated_img, rotated_target)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([270,480,3]),tf.TensorShape([48])))
train_dataset = train_dataset.batch(64).prefetch(1)
valid_dataset = tf.data.Dataset.from_generator(validGenerator, (tf.float32, tf.float32), (tf.TensorShape([270,480,3]),tf.TensorShape([48])))
valid_dataset = valid_dataset.batch(64).prefetch(1)

In [ ]:
# plt.imshow(a)
# plt.scatter(t[0::2], t[1::2], marker='x')

### Modeling

In [ ]:
earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.8, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./baseline_with_augmentation_rotation.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

callbacks = [earlystop, learning_rate_reduction, model_check]

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(270,480,3)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(48))
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
history = model.fit(train_dataset,
                    epochs=150,
                    validation_data=valid_dataset,
                    callbacks = callbacks,
                    verbose=1)

### Load Model & Fine Tuning

In [ ]:
saved_model = load_model('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/data/baseline_with_augmentation_rotation.h5')

model = Sequential()
for layer in saved_model.layers[:-4]: # go through until last layer
    model.add(layer)
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(48))
model.summary()

earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./baseline_with_augmentation_GAP.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

callbacks = [earlystop, learning_rate_reduction, model_check]

model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='mean_squared_error',
              metrics=['mae'])

history = model.fit(train_dataset,
                    epochs=150,
                    validation_data=valid_dataset,
                    callbacks = callbacks,
                    verbose=1)

In [ ]:
saved_model = load_model('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/data/baseline_with_augmentation_rotation.h5')

### Load Test Set & Predict

In [ ]:
test_paths = glob.glob('./test_imgs/*.jpg')
test_paths.sort()
X_test=[]

for test_path in tqdm(test_paths):
    img=tf.io.read_file(test_path)
    img=tf.image.decode_jpeg(img, channels=3)
    img=tf.image.resize(img, [270,480])
    img=img/255
    X_test.append(img)

X_test=tf.stack(X_test, axis=0)
X_test.shape

In [ ]:
# pred=model.predict(X_test)
pred=saved_model.predict(X_test)

### Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission.iloc[:,1:]=pred/8
submission

In [ ]:
bb = submission.iloc[1,1:49]
bb = np.array(bb)
aa = Image.open(test_paths[1])
plt.imshow(aa)
plt.scatter(bb[0::2], bb[1::2], marker='x')

In [ ]:
submission.to_csv('baseline_submission_saved.csv', index=False)